Здесь буду анализировать данные из разных открытых источников правительства по бюджету и подобным вещам.
Сейчас пока общие вопросы:
1. Анализ бюджетов текущего и прошлых годов (как менялся объём и распределение)
2. Источники пополнения бюджета
3. Доходы и расходы по регионам (какие регионы приносят больше всего доходов, какие меньше, как абсолютно так и на душу населения)
4. Параметры рождаемости и смертности
5. Инфляция
6. Данные о зарплатах по стране и регионам

## Источники данных
### Федеральная служба гос статистики (http://www.gks.ru/)

Содержит огромное количество данных, выпускаются различные итоговые документы за год, например:

Финансы России 2018 (http://www.gks.ru/free_doc/doc_2018/fin18.pdf)

### БД (http://cbsd.gks.ru/#)
Удобный инструмент просмотра статистики на сайте с возможность построения графиков


## Open data (http://www.gks.ru/opendata/dataset)
Непонятно что там есть

## Регионы России

http://www.gks.ru/free_doc/new_site/region_stat/sep_region.html

Здесь можно скачать справочник "Регионы России. Социально-экономические показатели", в котором много всего интересного

### Экономические показатели субъектов
Попробуем из pdf выше скопировать текст в текстовый файл и затем из этого файла сделать csv. Нужно учесть, что таблица разделена на 2 страницы, поэтому удобнее будет сделать 2 csv, один для одного набора колонок, другой для другого. На втором этапе нам нужно вытащить из этого списка федеральные округа при чтении. После чтения 2-х csv и фильтрации по округам, смёржим оба pandas объекта в один. Итак скопируем текст по первому набору колонок и создадим файл.

Тонкости:

Есть разные суммарные цифры и "в том числе", например "Архангельская область", в том числе "Ненецкий автономный округ", Архангельская область без автономного округа". Есть переносы строк в названии, нужно это всё править руками. Руами же удобнее выделить в отдельный документ федеральные округа. Хотя с другой стороны лучше попытаться всё автоматизировать, чтобы была возможность использовать этот парсер для даных за другие года.

Ещё местами к данным прикреплена скобка с цифрой и перед ней может быть ...

Есть ещё вот такие казусы: "24284,910140135,8 3643400", когда 2 числа слиплись, поэтому нужно в конце проверить, что во всех строках число колонок одинаковое и правильное.

У Питера вообще нет 3-х колонок во второй таблице!
Тут придётся тупо руками править(

In [59]:
import os
import os.path
import io
import pandas as pd

core_path = os.environ['GITHUB_PATH']
file_path1 = os.path.join(core_path, 'education/Statistics/Rosstat/reg_economy1.txt')
file_path2 = os.path.join(core_path, 'education/Statistics/Rosstat/reg_economy2.txt')

from locale import *
setlocale(LC_NUMERIC, '')

def check_columns_count(samples, num):
    for i, s in enumerate(samples):
        if len(s) != num:
            return i
    return -1

def num_there(s):
    return any(i.isdigit() for i in s)

def filter_parts(parts, is_name_first=True):
    real_parts = []
    last_is_num = True
    for i, p in enumerate(parts):
        p = p.strip(' \t\n\r')
        if not p:
            continue
            
        if not is_name_first and any(i.isalpha() for i in p):
            continue
            
        if p.endswith(')'):
            p = p[0:-2]
        
        if p == '...':
            p = '0'

        if num_there(p):
            val = float(p.replace(',', '.'))
            real_parts.append(val)
            last_is_num = True
        else:
            if i == 0 or last_is_num:
                real_parts.append(p)
            else:
                real_parts[-1] = real_parts[-1] + ' ' + p
            last_is_num = False
    return real_parts

def parse_datafile(file_path, is_name_first=True):
    lines = []
    with io.open(file_path, mode="r", encoding="utf-8") as f:
        curr_str = ''
        for l in f:
            if ':' in l:
                continue
                 
            s = l
            if curr_str:
                s = curr_str + ' ' + l
                curr_str = ''
                
            parts = s.split(' ')
            if len(parts) < 5: # part of name
                if is_name_first:
                    curr_str = s.strip(' \t\n\r')
                continue
            
            real_parts = filter_parts(parts, is_name_first)
            lines.append(real_parts)
            
    return lines
    
samples1 = parse_datafile(file_path1)
err_line_num = check_columns_count(samples1, 10) 
if err_line_num != -1:
    print('Part 1. Invalid number of columns in line ', err_line_num)
    print(samples1[err_line_num])
    
samples2 = parse_datafile(file_path2, False)

# Fix Piter
samples2[32] = [samples2[32][0], samples2[32][1], samples2[32][2], samples2[32][3], 0, 0, 0, samples2[32][4], samples2[32][5], samples2[32][6]]

err_line_num = check_columns_count(samples2, 10) 
if err_line_num != -1:
    print('Part 2. Invalid number of columns in line ', err_line_num)
    print(samples2[err_line_num])
    
headers1 = ['Name', 'Square', 'Population', 'Workers', 'MeanMonthIncome', 'MeanMonthCost', 'MeanMonthSalary', 'GDP', 'CapitalInvestments', 'Funds']
df1 = pd.DataFrame(samples1, columns=headers1)

headers2 = ['MineralsVolume', 'ManufactoreVolume', 'EnergyVolume', 'WaterVolume', 'AgricultureVolume', 'VegetableVolume', 'AnimalVolume', 'NewBuildingSquare', 'RetailOverturn', 'FinResult']
df2 = pd.DataFrame(samples2, columns=headers2)

df = df1.copy()
df[df2.columns] = df2[df2.columns]
df.head()

,Name,Square,Population,Workers,MeanMonthIncome,MeanMonthCost,MeanMonthSalary,GDP,CapitalInvestments,Funds,MineralsVolume,ManufactoreVolume,EnergyVolume,WaterVolume,AgricultureVolume,VegetableVolume,AnimalVolume,NewBuildingSquare,RetailOverturn,FinResult
0,Российская Федерация,17125.2,146880.4,71842.7,31422.0,23806.0,39167.0,69254134.3,15966804.0,194649464.0,1.39162e+07,38733173.0,5379682.0,1022498.0,5119844.0,2610174.0,2509670.0,79223.9,29813.3,9036848.0
1,Центральный федеральный округ,650.2,39311.4,21259.7,40843.0,30774.0,48593.0,24135019.0,4172962.0,60640167.0,1.50269e+06,13217385.0,1691479.0,312144.0,1304766.6,632213.0,672553.7,24284.9,10140135.8,3643400.0
2,Белгородская область,27.1,1549.9,757.9,30074.0,22073.0,29066.0,730562.0,139209.0,1501847.0,116588,629093.0,27031.0,10614.0,227267.4,69828.2,157439.2,1300.5,315389.9,138538.0
3,Брянская область,34.9,1211.0,530.2,26402.0,20629.0,24743.0,285847.9,54769.0,794829.0,277,191019.0,16064.0,7347.0,84470.0,31458.9,53011.1,558.3,234367.2,17432.0
4,Владимирская область,29.1,1378.3,640.6,23988.0,17813.0,26975.0,392051.8,79555.0,842074.0,4004,406484.0,33308.0,9987.0,28987.1,10672.5,18314.5,695.4,211869.5,32668.0


In [60]:
df_district = df[df.Name.str.contains('округ')]
df_russia = df[df.Name.str.contains('Федерация')]
df_russia

,Name,Square,Population,Workers,MeanMonthIncome,MeanMonthCost,MeanMonthSalary,GDP,CapitalInvestments,Funds,MineralsVolume,ManufactoreVolume,EnergyVolume,WaterVolume,AgricultureVolume,VegetableVolume,AnimalVolume,NewBuildingSquare,RetailOverturn,FinResult
0,Российская Федерация,17125.2,146880.4,71842.7,31422.0,23806.0,39167.0,69254134.3,15966804.0,194649464.0,1.39162e+07,38733173.0,5379682.0,1022498.0,5119844.0,2610174.0,2509670.0,79223.9,29813.3,9036848.0


In [61]:
df_subjects = df[~((df.Name.str.contains('округ') | df.Name.str.contains('Федерация')))]
df_subjects

,Name,Square,Population,Workers,MeanMonthIncome,MeanMonthCost,MeanMonthSalary,GDP,CapitalInvestments,Funds,MineralsVolume,ManufactoreVolume,EnergyVolume,WaterVolume,AgricultureVolume,VegetableVolume,AnimalVolume,NewBuildingSquare,RetailOverturn,FinResult
2,Белгородская область,27.1,1549.9,757.9,30074.0,22073.0,29066.0,730562.0,139209.0,1501847.0,116588,629093.0,27031.0,10614.0,227267.4,69828.2,157439.2,1300.5,315389.9,138538.0
3,Брянская область,34.9,1211.0,530.2,26402.0,20629.0,24743.0,285847.9,54769.0,794829.0,277,191019.0,16064.0,7347.0,84470.0,31458.9,53011.1,558.3,234367.2,17432.0
4,Владимирская область,29.1,1378.3,640.6,23988.0,17813.0,26975.0,392051.8,79555.0,842074.0,4004,406484.0,33308.0,9987.0,28987.1,10672.5,18314.5,695.4,211869.5,32668.0
5,Воронежская область,52.2,2333.8,1102.1,29327.0,23825.0,28007.0,841375.7,294169.0,1841669.0,6283,422944.0,88207.0,12116.0,193876.1,120821.1,73055.0,1687.3,516648.5,18249.0
6,Ивановская область,21.4,1014.6,456.3,24760.0,17000.0,23470.0,179633.4,27098.0,550729.0,755,133335.0,32003.0,4285.0,14909.9,5276.5,9633.4,339.8,156904.6,1594.0
7,Калужская область,29.8,1012.2,504.8,28108.0,20346.0,34332.0,373403.5,81301.0,989602.0,3672,672706.0,23200.0,8463.0,38538.4,17091.9,21446.5,882.3,186001.8,37477.0
8,Костромская область,60.2,643.3,290.8,24745.0,16660.0,24554.0,160705.4,20642.0,436507.0,317,118263.0,39628.0,3101.0,15993.0,5668.0,10325.1,310.2,97322.6,6315.0
9,Курская область,30.0,1115.2,519.6,26425.0,19343.0,27274.0,364601.7,100591.0,878321.0,67783,176100.0,60309.0,4582.0,130890.9,75706.9,55184.0,590.5,197188.3,56532.0
10,Липецкая область,24.0,1150.2,565.8,29294.0,22504.0,28455.0,470239.3,139900.0,1268311.0,5765,628043.0,25921.0,10347.0,101595.7,60542.9,41052.9,1084.2,236980.4,143885.0
11,Московская область,44.3,7503.4,3450.2,41286.0,30878.0,46836.0,3565258.0,678273.0,8044098.0,13870,2328071.0,279036.0,72279.0,87550.7,42942.6,44608.1,9077.7,2093604.5,400345.0


In [62]:
subjects_csv = os.path.join(core_path, 'education/Statistics/Rosstat/subjects.csv')
df_subjects.to_csv(subjects_csv, sep='\t', encoding='utf-8')

In [63]:
df_subjects.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 2 to 94
Data columns (total 20 columns):
Name                  81 non-null object
Square                81 non-null float64
Population            81 non-null float64
Workers               81 non-null float64
MeanMonthIncome       81 non-null float64
MeanMonthCost         81 non-null float64
MeanMonthSalary       81 non-null float64
GDP                   81 non-null float64
CapitalInvestments    81 non-null float64
Funds                 81 non-null float64
MineralsVolume        81 non-null object
ManufactoreVolume     81 non-null float64
EnergyVolume          81 non-null float64
WaterVolume           81 non-null float64
AgricultureVolume     81 non-null float64
VegetableVolume       81 non-null float64
AnimalVolume          81 non-null float64
NewBuildingSquare     81 non-null float64
RetailOverturn        81 non-null float64
FinResult             81 non-null float64
dtypes: float64(18), object(2)
memory usage: 13.3+ KB


In [67]:
# df_subjects['MineralsVolume']
df_subjects.sort_values('GDP')

,Name,Square,Population,Workers,MeanMonthIncome,MeanMonthCost,MeanMonthSalary,GDP,CapitalInvestments,Funds,MineralsVolume,ManufactoreVolume,EnergyVolume,WaterVolume,AgricultureVolume,VegetableVolume,AnimalVolume,NewBuildingSquare,RetailOverturn,FinResult
74,Республика Алтай,92.9,218.1,83.4,18411.0,11813.0,26316.0,46128.4,13111.0,131791.0,3593,4995.0,2822.0,356.0,10932.1,2199.4,8732.8,130.9,24371.4,19875.0
94,Еврейская автономная область,36.3,162.0,67.2,23386.0,16574.0,34409.0,46872.2,10455.0,252341.0,7593,6743.0,4209.0,446.0,5216.3,4273.2,943.1,50.4,22582.5,64.0
44,Республика Ингушетия,3.6,488.0,179.4,15131.0,7283.0,22750.0,50882.9,13587.0,119375.0,1463,1865.0,1345.0,487.0,9031.3,1952.7,7078.7,309.3,22500.8,-1465.0
76,Республика Тыва,168.6,321.7,98.7,14048.0,7657.0,31251.0,52221.3,9342.0,99573.0,23347,370.0,4669.0,326.0,5710.0,904.9,4805.2,101.0,22139.9,6732.0
35,Республика Калмыкия,74.7,275.4,111.1,14730.0,7940.0,22919.0,56045.1,10448.0,203657.0,…,1235.0,2054.0,395.0,27084.3,4974.6,22109.7,96.3,19631.9,745.0
41,г. Севастополь,0.9,436.7,182.1,24713.0,20666.0,27687.0,64163.2,30382.0,304437.0,…,13310.0,6492.0,1729.0,2474.3,2279.1,195.2,174.3,63290.1,620.0
46,Карачаево-Черкесская Республика,14.3,466.3,169.2,17142.0,9429.0,22638.0,73151.3,17812.0,210594.0,2751,35194.0,9061.0,991.0,27468.6,12924.8,14543.8,201.8,37075.6,904.0
34,Республика Адыгея,7.8,453.4,152.1,24677.0,19245.0,24490.0,91352.4,22858.0,202111.0,2740,47823.0,2346.0,1807.0,21612.8,12310.1,9302.7,244.3,86249.4,405.0
47,Республика Северная Осетия - Алания,8.0,701.8,289.7,22773.0,17039.0,24715.0,125498.3,26807.0,271077.0,661,27588.0,6697.0,1542.0,21926.6,9581.7,12344.9,192.2,109722.5,-362.0
45,Кабардино-Балкарская Республика,12.5,865.8,362.6,20385.0,15423.0,22782.0,132706.9,40875.0,272146.0,398,27688.0,8733.0,1069.0,45570.2,24745.7,20824.6,426.8,123965.1,-801.0


## Платёжный баланс

# Демография

## Естественный прирост

Естественный прирост не учитывает миграцию, только рождаемость и смертность, для независимости от общей численности населения страны, удобно брать показатели на 1000 человек

Возьмём таблицу "
http://www.gks.ru/wps/wcm/connect/rosstat_main/rosstat/ru/statistics/population/demography/#